# Cálculo de subnotificação

Primeiro, importando os pacotes

In [1]:
import pandas as pd
from scipy.stats import nbinom
import scipy
import csv
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Definindo algumas constantes e importanto bancos de dados

In [2]:
# Aqui, definem-se as faixas de idade que serao utilizadas
faixaEtaria = ["from_0_to_9", "from_10_to_19", "from_20_to_29", "from_30_to_39", "from_40_to_49", "from_50_to_59", "from_60_to_69", "from_70_to_79", "from_80_to_older"]

# Define-se a mortalidade referente a cada faixa etaria
mortalidadeFaixa = {"from_0_to_9": 0.00002,
                    "from_10_to_19": 0.00006,
                    "from_20_to_29": 0.0003,
                    "from_30_to_39": 0.0008,
                    "from_40_to_49": 0.0015,
                    "from_50_to_59": 0.006,
                    "from_60_to_69": 0.022,
                    "from_70_to_79": 0.051,
                    "from_80_to_older":   0.093}

# Importa o arquivo de dados
dfDadosCasos = pd.read_csv("br_health_region_cases_history.csv")
dfDadosCasos.drop(dfDadosCasos.columns[0], axis=1,inplace=True)

dfRegioes = pd.read_csv("br_health_region_tabnet_age_dist_2019_treated.csv")


# Obtem a lista de cidades presentes no arquivo
listaCidades = list(dfRegioes["health_region_name"].unique())
codigosCidades = list(dfRegioes["health_region_id"].unique())

dfDadosCasos

,health_region_id,last_updated,confirmed_cases,deaths
0,11001,2020-04-10,2.0,0.0
1,11001,2020-04-11,2.0,0.0
2,11001,2020-04-12,7.0,0.0
3,11001,2020-04-13,7.0,0.0
4,11001,2020-04-14,7.0,0.0
...,...,...,...,...
48399,53001,2020-07-16,68952.0,945.0
48400,53001,2020-07-17,70414.0,967.0
48401,53001,2020-07-18,71734.0,979.0
48402,53001,2020-07-19,72892.0,989.0


Inicializando os dicionários a serem utilizados

In [3]:
# Dicionario para armazenar a populacao de cada cidade por faixa etaria
populationByAge = {}
# Dicionario para armazenar a populacao total de cada cidade
totalPopulation = {}
# Dicionario para armazenar a proporcao da populacao de cada faixa etaria frente a populacao total
populationProportion = {}
# Dicionario para armazenar o produto da mortalidade pela populacao de cada faixa etaria por cidade
mortalidadeCidadeFaixa = {}
# Dicionario para armazenar o valor referente a mortalidade geral por cidade
mortalidadeGeralCidade = {}

Preenchendo os dicionários criados

In [4]:
# Novamente varrendo a lista de cidades
for codigo in codigosCidades:
    # Inicializando a variavel que contera a populacao total da cidade
    total = 0
    # Para cada uma das faixas etarias
    for faixa in faixaEtaria:
        # Constroi uma chave que contem a cidade e a faixa analisada
        key = str(codigo) + "_" + faixa
        # Separa as linhas referentes a uma dada cidade
        row = dfRegioes.loc[dfRegioes["health_region_id"] == codigo]
        # Separa a primeira linha das obtidas acima
        row = row.head(1)
        # Armazena o valor da populacao de uma dada faixa etaria para uma cidade
        populationByAge[key] = row[faixa].item()
        # Soma para obter o total da populacao de uma cidade
        total += row[faixa].item()
    # Armazena o valor da populacao total
    totalPopulation[codigo] = total
    
    # Varrendo novamente as faixas etarias
    for faixa in faixaEtaria:
        # Constroi uma chave que contem a cidade e a faixa analisada
        key = str(codigo) + "_" + faixa
        # Calcula e armazena a proporcao de uma dada faixa etaria
        populationProportion[key] = populationByAge[key] / total
        # Calcula e armazena a mortalidade por faixa de uma dada cidade
        mortalidadeCidadeFaixa[key] = mortalidadeFaixa[faixa] * populationProportion[key]
        
    # Inicializa variavel que contera a taxa de mortalidade geral de uma cidade
    totalMortalidadeCidade = 0
    # Varre cada uma das faixas etarias
    for faixa in faixaEtaria:
        # Constroi uma chave que contem a cidade e a faixa analisada
        key = str(codigo) + "_" + faixa
        # Soma-se ao valor atual da mortalidade geral para a cidade analisada
        totalMortalidadeCidade += mortalidadeCidadeFaixa[key]
        
    # Armazena a taxa de mortalidade geral para uma cidade
    mortalidadeGeralCidade[codigo] = totalMortalidadeCidade
    
mortalidadeGeralCidade

{11001: 0.004345116949251466,
 11002: 0.005289083977894131,
 11003: 0.005480903200986264,
 11004: 0.003828609748668121,
 11005: 0.005313912877087365,
 11006: 0.004784322731211222,
 11007: 0.004337298475809169,
 12001: 0.004405500286998277,
 12002: 0.004075751067224279,
 12003: 0.003248188195957847,
 13001: 0.003911913531147893,
 13002: 0.0034252177386062716,
 13003: 0.0035784050819351832,
 13004: 0.004022869413920473,
 13005: 0.0038665704970889747,
 13006: 0.003655816665158234,
 13007: 0.0029920213615057542,
 13008: 0.002961140655845142,
 13009: 0.0030144160211540215,
 14001: 0.00333532239224163,
 14002: 0.0033222788871065875,
 15001: 0.003622523037611923,
 15002: 0.004468737007833202,
 15003: 0.0035085260593762566,
 15004: 0.003248349520031716,
 15006: 0.005582182920292315,
 15007: 0.004338371836166184,
 15008: 0.004299782078535609,
 15009: 0.004534271160699934,
 15010: 0.0038791483030727837,
 15011: 0.0039239203845188335,
 15012: 0.0037673692745149538,
 15013: 0.0039411677396353675,


Define a funcao que executara a simulacao

In [5]:
# Funcao que retorna o valor de uma distribuicao binomial negativa frente aos parametros dados
def simulacao(numeroSimulacoes, chanceAcerto, k, p):
    a = (100-chanceAcerto)
    b=chanceAcerto+(100-chanceAcerto)
    r = nbinom.rvs(k,p, size=numeroSimulacoes)
    m = scipy.stats.tmean(r,(scipy.stats.scoreatpercentile(r,a),scipy.stats.scoreatpercentile(r,b)))
    # Retorna o inteiro do valor mais provável retornado pela distribuicao
    return int(m)

Para cada cidade, realiza a simulação com parametros numeroSimulacoes = 1000000, chanceAcerto = 99, k = 3, p = mortalidadeGeralCidade (para cada cidade).

In [6]:
# Define parametros para as simulacoes
numeroSimulacoes = 10000
chanceAcerto = 99
sizeOfSlidingWindow = 15
daysBefore = 19

# O trecho a seguir calcula a soma de mortes com a janela deslizante para cada cidade
# Associa o resultado com uma data 'daysBefore' anterior

# Inicializa um novo DataFrame
finalDadosCovid = pd.DataFrame(columns=dfDadosCasos.columns)

# Varre todas as cidades
for codigo in codigosCidades:
    # Separa as linhas referentes a uma dada cidade
    rows = dfDadosCasos.loc[dfDadosCasos["health_region_id"] == codigo]
    # Converte a data para o formato padrao
    rows["last_updated"] = pd.to_datetime(rows['last_updated'], format='%Y-%m-%d')
    # Ordena os valores por data
    rows = rows.sort_values(by="last_updated")
    # Obtem o numero de mortes acumuladas dentro de uma janela de tamanho definido acima (sizeOfSlidingWindow)
    #rows["accumulatedDeathsWindow"] = rows["dailyDeaths"].rolling(sizeOfSlidingWindow, min_periods=1).sum()
    # Volta "daysBefore" dias na data analisada (atraso entre contaminacao e confirmacao)
    rows["dateInfected"] = rows["last_updated"] - datetime.timedelta(days=daysBefore)
    rows["dateInfected"] = rows["dateInfected"]
    # Separa os dados para insercao
    frames = [finalDadosCovid, rows]
    # Insere no DataFrame final
    finalDadosCovid = pd.concat(frames,ignore_index=True)

# Salva o dataframe contendo dados estimados de contaminados e a data correta em um csv
finalDadosCovid.to_csv("dados_com_contaminados.csv")

Realiza para cada valor de mortes acumuladas a simulação para estimar o número de infectados

In [7]:
# Inicializa lista que contera os valores de estimativas diarias de contaminacao
infectedList = []
# Passa por toda base
for index, rows in finalDadosCovid.iterrows():
    # Realiza a simulacao para obter o valor estimado de contaminados
    if rows["deaths"] > 0:
        infectedList.append(simulacao(numeroSimulacoes, chanceAcerto, rows["deaths"], mortalidadeGeralCidade[rows["health_region_id"]]))
    # Caso o numero de mortes seja 0, retorna 0
    else:
        infectedList.append(np.nan)

# Insere os valores calculados
finalDadosCovid["estimatedInfected"] = infectedList

Salva o conteúdo calculado em um arquivo .csv

In [8]:
# Esse arquivo contera apenas a cidade, a data e o numero de infectados estimados
with open('contaminados_por_data.csv', 'w', newline='') as finalFile:
    writer = csv.writer(finalFile, delimiter=",")
    writer.writerow(["health_region_id", "date", "infected"])
    # Escreve no arquivo a cidade, a data e o numero de infectados estimados
    for index, rows in finalDadosCovid.iterrows():
        frame = [rows["health_region_id"], rows["dateInfected"], rows["estimatedInfected"]]
        writer.writerow(frame)

In [ ]:
# Aqui, serao removidos registros de cidades que apresentaram zero contaminados em uma determinada data
dfLimpo = pd.read_csv("contaminados_por_data.csv")
dfLimpo = dfLimpo.dropna()

# Em seguida, sera criado um novo arquivo que contera a subnotificacao por data para uma determinada cidade
with open('subnotificacoes_raw.csv', 'w', newline='') as finalFile:
    writer = csv.writer(finalFile, delimiter=",")
    # Cabecalho do arquivo
    writer.writerow(["health_region_id", "date","estimated_infected", "confirmed_by_gov", "subnotification"])
    # Percorre todo banco
    for index, row in dfLimpo.iterrows():
        # Varre todas as cidades
        for codigo in codigosCidades:
            # Separa os dados referentes a determinada cidade
            rows = finalDadosCovid.loc[finalDadosCovid["health_region_id"] == codigo]
            # Separa os dados referentes a determinada data (considerando o atraso entre contaminacao e deteccao)
            rows = rows.loc[rows["dateInfected"] == row["date"]]
            # Se obtivermos resultados apos a selecao (pois podem nao existir registros na data que estimamos)
            if not rows.empty:
                # Calculamos a subnotificacao, caso o numero confirmado pelo governo seja maior que zero
                try:
                    fraction = row["infected"]/rows["confirmed_cases"].astype(float).values[0]
                    # Escrevendo no arquivo
                    frame = [rows["health_region_id"].values[0] , rows["dateInfected"].values[0], row["infected"], rows["confirmed_cases"].astype(float).values[0], fraction]
                    writer.writerow(frame)
                # Caso seja zero, a principio, a subnotificacao e infinita
                except ZeroDivisionError:
                    # Escrevendo no arquivo
                    frame = [rows["health_region_id"].values[0], rows["dateInfected"].values[0], row["infected"], rows["confirmed_cases"].astype(float).values[0], "inf"]
                    writer.writerow(frame)

In [ ]:
# Finalmente, sera calculada a media dos valores de subnotificacao para uma dada cidade
# Primeiro, carrega-se o arquivo gerado na etapa anterior
dfSubnotificacoes = pd.read_csv("subnotificacoes_raw.csv")
with open('subnotificacoes.csv', 'w', newline='') as finalFile:
    writer = csv.writer(finalFile, delimiter=",")
    # Escreve-se o cabecalho
    writer.writerow(["city", "subnotification"])
    # Percorre a lista de cidades
    for codigo in codigosCidades:
        # Separa-se as linhas referentes a uma dada cidade
        rows = dfSubnotificacoes.loc[dfSubnotificacoes["health_region_id"] == codigo]
        # Agora, limpa-se os registros que indicam subnotificacao infinita (distorceriam a media)
        rows['subnotification'] = rows['subnotification'].astype(str)
        rows = rows[~rows['subnotification'].str.contains("inf", regex=True)]
        rows['subnotification'] = rows['subnotification'].astype(float)
        # Calcula-se a media
        mean = rows["subnotification"].mean()
        # Prepara os dados para insercao
        frame = [cidade, mean]
        # Insere os dados no arquivo final (subnotificacoes.csv)
        writer.writerow(frame)
        